# 1. Lorenz-96を4次のRunge-Kutta法を用いて実装する。パラメータ値Fをいろいろ変え、F=8の時にカオスとなることを確認する。余裕があれば、他の時間積分スキームも実装してみる。

## 計算の準備
初期値: 
$ X_j = F $とし，摂動として$ X_{20} $に$ 1.001 $をかける．
$ X_{20} = F \cdot 1.001$

変数:
$J=40$

時間ステップ:
$N=360 \times 20$
(天下り的ではあるが１年分)

カオスの定義:
- 周期性がない(視認)
- 課題2でのリヤプノフ指数$\lambda$を計算しnot zero

以下の2種類のplotを行う．
- 摂動を加えた20番目の要素をplot
- 1,2,3番目の要素を3次元plot

In [ ]:
%matplotlib inline
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import numpy as np

# モジュールの読み込み
import sys
sys.path.append('./module')
from utils import make_lorenz96, rk4, euler

In [ ]:
def plot_lorenz96(result):
    result = np.array(result)

    # 摂動を加えた20番目の要素をplot
    fig1, ax1 = plt.subplots()
    ax1.plot(result[:, 19])
    ax1.set_xlabel('step')
    ax1.set_ylabel('$x_{20} $')
    plt.title('plot 20th component')

    # 1,2,3番目の要素を3次元plot
    fig2 = plt.figure()
    ax2 = fig2.gca(projection='3d')
    ax2.plot(result[:, 0], result[:, 1], result[:, 2])
    ax2.set_xlabel('$x_1$')
    ax2.set_ylabel('$x_2$')
    ax2.set_zlabel('$x_3$')
    plt.title('first three components of Lorenz96')
    
    # Hovmoller Diagram
    day = 50
    Z = result[180*20:180*20 + day*20, :]
    
    fig3, ax3 = plt.subplots(figsize=(5,5))
    ax3.grid(False)
    ax3.set_xlabel('space')
    ax3.set_ylabel('time(day)')
    ax3.set_xticks(np.arange(0,40,5))
    ax3.set_yticks(np.arange(day*20, 0, -20*5))
    ax3.set_yticklabels(np.arange(180, 180+ day, 5))
    im = ax3.imshow(Z, aspect=40/(day*20), extent=[0, 40, 0, day*20], vmax=Z.max(), vmin=Z.min())
    plt.colorbar(im)
    ax3.set_title('Hovmollor diagram')
    
    plt.show()

In [ ]:
# J: 変数の数
J = 40

# dt: 時間刻み
dt = 0.01

# N: 時間ステップ数
# 天下り的ではあるが1年分に相当
N = 360*20

# 任意のスキームでLorenz96を計算，plotする．
# 引数: scheme(t, x, dt, f) -> df
def simulate_lorenz96(scheme, plot=True):
    result = np.zeros((N,J))
    x = x0
    result[0] = x[:]

    for n in range(1,N):
        t = n*dt
        x = scheme(t, x, dt, lorenz)
        result[n] = x[:]
        
#  plot
    if plot:
        plot_lorenz96(result)

    return result

## 4次RungeKuttaでの計算結果

$F=8$

In [ ]:
# F = 8
F = 8 # 外力

# 初期値の設定(摂動を加える)
x0 = F*np.ones(J)
x0[19] *= 1.001

lorenz = make_lorenz96(F)
x = simulate_lorenz96(rk4)

# F = 4
F = 4
x0 = F*np.ones(J)
x0[19] *= 1.001
lorenz = make_lorenz96(F)

_ = simulate_lorenz96(rk4)

# F = 1
F = 1
x0 = F*np.ones(J)
x0[19] *= 1.001
lorenz = make_lorenz96(F)

_ = simulate_lorenz96(rk4)

# 2. パラメータ値F=8とする。誤差の平均発達率について調べ、0.2時間ステップを1日と定義することの妥当性を確認する。

### 方針
- rmseの平均発達率を調べる．
- アトラクター上の点をとり，ガウシアンノイズを加えた軌道との誤差の時間発展の平均を取る．

平均の取り方: 以下の2方向の平均をとる
- アトラクター平均
- 誤差の方向の平均をとる(atrと直交する方向にずらすとatrに近づく傾向にある)
<!-- - リヤプノフ指数を数値的に近似計算することによって誤差の平均発達率を評価する． -->

$F=8$

$dt=0.01$

$step=7 \cdot 20$


## 結果

In [ ]:
# 外力
F=8

lorenz = make_lorenz96(F)

day = 4

# step数
# 天下り的ではあるが１週間分
N_step = day*20

# atrからのサンプル数
N_atr_sample = 180//day

# ノイズについてのサンプル数
N_noise_sample = 5

# 時間刻み幅
dt = 0.01

# 摂動の大きさ
epsilon = 1

error = np.zeros((N_atr_sample, N_step))
np.random.seed(1)

for m in range(N_atr_sample):
#     半分以降の軌道からサンプル
    z0 = x[180*20+day*20*m, :]
    z = np.zeros((N_step, J))
    z[0] = z0
    
    temp_error = np.zeros((N_noise_sample, N_step))
    
    for i in range(N_noise_sample):
        z_perturb = z[0] + epsilon*np.random.normal(size=z[0].shape)
        temp_error[i, 0] = np.linalg.norm(z[0] - z_perturb, ord=2)/np.sqrt(J)
    
        for n in range(1, N_step):
            t = n*dt
            if i == 0:
                z[n] = rk4(t, z[n-1], dt, lorenz)
            z_perturb = rk4(t, z_perturb, dt, lorenz)
            temp_error[i, n] = np.linalg.norm(z[n] - z_perturb, ord=2)/np.sqrt(J)
            
    error[m] = temp_error.mean(axis=0)[:]

error_mean = error.mean(axis=0)

# plot
fig, ax = plt.subplots()
ax.plot(error_mean)
ax.set_xlabel('steps')
ax.set_ylabel('RMSE')
plt.grid()
_ = plt.title('RMSE grow up')

# logを取ったものもplot
fig1, ax1 = plt.subplots()
ax1.plot(np.log(error_mean))
plt.grid()
_ = plt.title('log error grow up')

## 観察
- 最初，誤差が1より小さくなっているがこれは摂動を与えた$x'$がattractorに引き寄せられるのが原因．

## 1から50step後の誤差の発達率を計算

In [ ]:
# duration = 40
error_mean_duration = np.array([
    np.array([error_mean[i+duration]/error_mean[i] for i in range(N_step-duration)]).mean()
    for duration in range(1,51)])

fig, ax = plt.subplots()
ax.plot(error_mean_duration)
ax.plot(2*np.ones(50))
ax.set_xlabel('duration steps')
ax.set_ylabel('ratio')
plt.grid()
_ = plt.title('error grow up ratio after duration steps')

### 考察
- 約40回(37回くらい)ごとに誤差が2倍になっている．
- 論文によると気象学では2日で誤差が2倍になるとある．

よって$ dt = 0.01 $なので
$ 40 \times 0.01 = 2 day $から
$ 1 day = 0.2 $
と考える．

### 誤差の収束
時間的に十分離れたattractor上の2点をとり誤差を調べる．

おおよそ5に近づく


In [ ]:
N_atr_sample = 180
long_errors = np.zeros(N_atr_sample)
for m in range(N_atr_sample):
    z1 = x[20*m, :]
    z2 = x[180*20+20*m, :]
    long_errors[m] = np.linalg.norm(z1-z2, ord=2)/np.sqrt(40)
print(long_errors.mean())
_ = plt.plot(long_errors)